# Understand My Requirements & Architecture — Without Being a SysML Expert

**Use case:** "I have a `.sysml` file in Istari and I just want to know what's in it."

This notebook connects to Istari, finds a SysML v2 model, runs SysGit extraction to produce
structured data and diagrams, then displays the results — all without installing a SysML editor.

**What you'll get:**
- 11 requirements (range, weight, speed, payload, safety, etc.) with target values
- 39 parts (propulsion, power, flight control, comms, payload, airframe) with attributes
- A requirements hierarchy diagram
- A parts block diagram

**Example model:** Group 3 UAS — a notional expendable tailless flying wing UAV.

See [`example-output/`](example-output/) for what the results look like without running anything.

In [ ]:
# Setup — install SDK and connect to Istari
import sys, os
from pathlib import Path

# Install SDK if needed (e.g. in Colab)
try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

# Add repo root to path so we can import istari_client
repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Find the model
#
# Replace MODEL_ID with your own, or use the default Group3 UAS SysML model.
# Find model IDs by running: python getting-started/01_explore_systems.py

MODEL_ID = "22bb5f2d-b902-4418-a75c-9e1ae1504c53"  # Group3 UAS Requirements

model = client.get_model(MODEL_ID)
print(f"Model: {model.display_name or 'Group3 UAS Requirements'}")
print(f"  File ID: {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    print(f"    - {rev.name} ({rev.size:,} bytes)")
print(f"  Existing artifacts: {len(model.artifacts)}")

# Expected output:
#   Model: Group3 UAS Requirements
#     File ID: 2ab0a2cf-4db6-4d97-b0d3-31e50b75905d
#     Revisions: 2
#       - group3_uas.sysml (22,681 bytes)
#       - Group3-UAS-Requirements-downloaded.sysml (22,681 bytes)
#     Existing artifacts: 4

In [ ]:
# Peek at the raw SysML text
#
# This is what the file looks like — SysML v2 syntax with requirements, parts,
# constraints, and satisfy statements all interleaved. 596 lines of text.

content = model.read_text()
lines = content.splitlines()
print(f"Total: {len(lines)} lines, {len(content):,} characters\n")
print("--- First 40 lines ---")
for i, line in enumerate(lines[:40], 1):
    print(f"{i:4d} | {line}")

# You'll see the package declaration, requirement definitions with attributes,
# and part definitions — but it's hard to quickly answer "what requirements exist?"
# or "what are the key parts?" from raw text. That's what extraction is for.

In [ ]:
# Run SysGit extraction
#
# Submits a job that parses the SysML file and produces 4 artifacts:
#   - output_requirements.json   (structured requirement data)
#   - output_parts.json          (structured parts data)
#   - requirements_hierarchy.png (visual requirements tree)
#   - parts_diagram.png          (visual parts block diagram)
#
# Takes ~1-3 minutes depending on agent availability.
# If artifacts already exist from a previous run, you can skip this cell.

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract_sysmlv2",
    tool_name="sysgit",
    tool_version="0.1.8",
    operating_system="Ubuntu 22.04",
    parameters={},
)
print(f"Job: {job.id}")

# Poll until done
while True:
    sleep(5)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nExtraction complete!")
    model = client.get_model(MODEL_ID)  # refresh to see new artifacts
    print(f"Artifacts ({len(model.artifacts)}):")
    for a in model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        size = f"{rev.size:,} bytes" if rev else "?"
        print(f"  - {rev.name if rev else a.name} ({size})")
else:
    print("\nExtraction failed!")
    if job.status_history:
        for s in job.status_history:
            print(f"  {s.name}: {getattr(s, 'message', '')}")

In [ ]:
# View requirements
#
# The extraction produces a dict keyed by qualified SysML name.
# Each requirement has: name, description, parent reference, and attributes
# (target values, units, priority).
#
# Expected: 11 requirements including RangeReq (1500 nm), MaxStructureWeight (275 lb),
# CruiseSpeed (100 knots), PayloadCapacity (125 lb), FailSafeReq, etc.

import json
from IPython.display import HTML, display

# Refresh model if needed (in case you skipped the extraction cell)
model = client.get_model(MODEL_ID)

# Find the requirements artifact
reqs_artifact = None
for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "requirements" in rev.name and rev.name.endswith(".json"):
        reqs_artifact = a
        break

if reqs_artifact:
    reqs_data = json.loads(reqs_artifact.read_text())
    print(f"Found {len(reqs_data)} requirements\n")
    
    # Build HTML table from the dict
    rows = ""
    for qname, r in reqs_data.items():
        name = r.get("name", "—")
        desc = r.get("description", "—")
        attrs = r.get("attributes", {})
        
        # Format attributes — show the "shall" statement and key values
        attr_parts = []
        for k, v in attrs.items():
            if k == "description":
                continue  # skip duplicate of the shall statement
            attr_parts.append(f"{k}: {v}")
        attr_str = ", ".join(attr_parts) if attr_parts else "—"
        
        # Get the shall statement from attributes if present
        shall = attrs.get("description", desc)
        
        rows += f'<tr><td><b>{name}</b></td><td>{shall}</td><td>{attr_str}</td></tr>\n'
    
    html = f"""
    <table style="border-collapse: collapse; width: 100%; font-size: 13px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1; white-space: nowrap;">Requirement</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Shall Statement</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Values</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>
    """
    display(HTML(html))
else:
    print("Requirements artifact not found. Run the extraction cell first.")

In [ ]:
# View parts
#
# The parts JSON is a dict keyed by qualified name. Each part has:
#   - declared_name: short name (e.g. "engine", "gps", "wing")
#   - attributes: dict of name → {name, value, code, type}
#
# Expected: 39 parts across 6 subsystems (Propulsion, Power, Flight Control,
# Communication, Payload, Airframe)

parts_artifact = None
for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "parts" in rev.name and rev.name.endswith(".json"):
        parts_artifact = a
        break

if parts_artifact:
    parts_data = json.loads(parts_artifact.read_text())
    print(f"Found {len(parts_data)} parts\n")
    
    rows = ""
    for qname, p in parts_data.items():
        name = p.get("declared_name", qname.split("::")[-1])
        attrs = p.get("attributes", {})
        
        # Show attributes with values (skip unset ones for readability)
        attr_parts = []
        for aname, ainfo in list(attrs.items())[:6]:
            val = ainfo.get("value")
            if val is not None:
                attr_parts.append(f"{aname}: {val}")
            else:
                attr_parts.append(f"{aname}: <i>unset</i>")
        attr_str = "<br>".join(attr_parts)
        if len(attrs) > 6:
            attr_str += f"<br><i>+{len(attrs) - 6} more</i>"
        
        # Indent sub-parts based on :: depth
        depth = qname.count("::") - 1
        indent = "&nbsp;&nbsp;&nbsp;&nbsp;" * max(0, depth - 1)
        
        rows += f'<tr><td>{indent}<b>{name}</b></td><td>{attr_str or "—"}</td></tr>\n'
    
    html = f"""
    <table style="border-collapse: collapse; width: 100%; font-size: 13px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1; white-space: nowrap;">Part</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Attributes</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>
    """
    display(HTML(html))
else:
    print("Parts artifact not found. Run the extraction cell first.")

In [ ]:
# View diagrams
#
# requirements_hierarchy.png — tree showing TopLevelRequirements branching to
#   10 child requirements (RangeReq, MaxStructureWeight, CruiseSpeed, etc.)
#
# parts_diagram.png — block diagram showing the Drone system with 6 subsystems
#   and all 39 parts with their key attribute values

from IPython.display import Image, display, Markdown

for label, keyword in [("Requirements Hierarchy", "requirements"), ("Parts Diagram", "parts")]:
    png_artifact = None
    for a in model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        if rev and keyword in rev.name and rev.name.endswith(".png"):
            png_artifact = a
            break
    
    if png_artifact:
        display(Markdown(f"### {label}"))
        img_bytes = png_artifact.read_bytes()
        display(Image(data=img_bytes))
    else:
        print(f"{label}: not found")

## Summary

From a single `.sysml` file (596 lines of SysML v2 text), the extraction produced:

| Artifact | Contents |
|----------|----------|
| `output_requirements.json` (5.5 KB) | 11 requirements — range, weight, speed, payload, temperature, safety, position, video, control |
| `output_parts.json` (34.6 KB) | 39 parts across 6 subsystems — each with typed attributes and values |
| `requirements_hierarchy.png` | Visual tree: TopLevelRequirements → 10 child requirements |
| `parts_diagram.png` | Block diagram: Drone → Propulsion, Power, Flight Control, Comms, Payload, Airframe |

### Key requirements found:

| Requirement | Target |
|-------------|--------|
| Range | 1,500 nm |
| Max Structure Weight | 275 lb |
| Cruise Speed | 100 knots |
| Payload Capacity | 125 lb |
| Operating Temperature | -10°C to 45°C |
| Position Accuracy | 2.5 m |

## What's Next?

- **Edit a requirement** and re-extract — see [`sysgit/`](../../sysgit/)
- **Run an nTop wing design** using these requirements as targets — see [`ntop/`](../../ntop/)
- **Share the extracted artifacts** with your team — see [`getting-started/03_share_resources.py`](../../getting-started/03_share_resources.py)
- **Upload a new version** of the .sysml and re-extract to compare — see [`getting-started/02_version_model.py`](../../getting-started/02_version_model.py)